In [1]:
import dvc.api
import pandas as pd

In [32]:
df1 = pd.read_csv(dvc.api.get_url(
    "data/wish_listing_label/processed/Appen_First_Delivery_20221212.csv",
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
))
df2 = pd.read_csv(dvc.api.get_url(
    "data/wish_listing_label/processed/Appen_Second_Delivery_20221216.csv",
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
))

In [33]:
df_tax = pd.read_json(dvc.api.get_url(
    "data/taxonomy/wish_newtax.json",
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [34]:
leafpath2id = {}
for i in df_tax.to_dict('records'):
    if i['is_leaf']:
        leafpath2id[i['category_path']] = int(i['id'])

In [66]:
leafid2path = {leafpath2id[i]: i for i in leafpath2id}

In [35]:
df = df2

In [36]:
df['labeler_category_path_leafid'] = df.labeler_category_path.apply(lambda x: leafpath2id[x] if x in leafpath2id else -1)

In [38]:
df['appen_category_path_leafid'] = df.appen_category_path.apply(lambda x: leafpath2id[x] if x in leafpath2id else -1)

In [42]:
df_ = df[(df.labeler_category_path_leafid != -1) & (df.appen_category_path_leafid != -1)]
len(df_) / len(df)

0.9956140350877193

In [43]:
df = df_

In [44]:
(df.labeler_category_path_leafid == df.appen_category_path_leafid).mean()

0.6051644952666604

In [46]:
from cleanlab.dataset import health_summary

In [61]:
leafid2labid = {}
for i in df_tax[df_tax.is_leaf]['id'].to_list():
    if i not in leafid2labid: 
        leafid2labid[i] = len(leafid2labid)

In [62]:
labid2leafid = {leafid2labid[i]: i for i in leafid2labid}

In [63]:
df.labeler_category_path_labid = df.labeler_category_path_leafid.apply(lambda x: leafid2labid[x])
df.appen_category_path_labid = df.appen_category_path_leafid.apply(lambda x: leafid2labid[x])

/tmp/ipykernel_26848/2957247474.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.labeler_category_path_labid = df.labeler_category_path_leafid.apply(lambda x: leafid2labid[x])
/tmp/ipykernel_26848/2957247474.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.appen_category_path_labid = df.appen_category_path_leafid.apply(lambda x: leafid2labid[x])


In [67]:
class_names = []
for i in range(len(labid2leafid)):
    class_names.append(leafid2path[labid2leafid[i]])

In [71]:
import numpy as np

In [82]:
pred_probs_appen = np.zeros((len(df), len(class_names)))
pred_probs_appen[np.arange(len(df)), df.appen_category_path_labid] = 1

pred_probs_offshore = np.zeros((len(df), len(class_names)))
pred_probs_offshore[np.arange(len(df)), df.labeler_category_path_labid] = 1


In [93]:
report_assume_offshore2appen = health_summary(labels=df.labeler_category_path_labid, pred_probs=pred_probs_appen, class_names=np.array(class_names),
    asymmetric=False
)

---------------------------------------------------------------
|  Generating a Cleanlab Dataset Health Summary               |
|   for your dataset with 64,014 examples and 5,291 classes.  |
|  Note, Cleanlab is not a medical doctor... yet.             |
---------------------------------------------------------------



/opt/conda/envs/py38/lib/python3.8/site-packages/cleanlab/dataset.py:90: RuntimeWarning: invalid value encountered in divide
  given_conditional_noise = given_label_noise / joint.sum(axis=1)  # p(y!=k, s=k) / p(s=k)
/opt/conda/envs/py38/lib/python3.8/site-packages/cleanlab/dataset.py:91: RuntimeWarning: invalid value encountered in divide
  true_conditional_noise = true_label_noise / joint.sum(axis=0)  # p(s!=k, y=k) / p(y=k)


Overall Class Quality and Noise across your dataset (below)
------------------------------------------------------------ 



,Class Name,Class Index,Label Issues,Inverse Label Issues,Label Noise,Inverse Label Noise,Label Quality Score
0,Beauty & Health > Bath & Shower > Bath,2191,1,1,1.0,1.0,0.0
1,Home & Garden > Household Cleaning > Household...,1273,1,1,1.0,1.0,0.0
2,Automobiles & Motorcycles > Car Repair Tools >...,3415,1,0,1.0,NaN,0.0
3,Home & Garden > Household Cleaning > Household...,1268,1,0,1.0,NaN,0.0
4,Automobiles & Motorcycles > Car Wash & Mainten...,3473,1,0,1.0,NaN,0.0
...,...,...,...,...,...,...,...
5286,Women's Clothing > Underwear & Sleepwear > Wom...,5286,0,13,NaN,1.0,NaN
5287,Women's Clothing > Underwear & Sleepwear > Wom...,5287,0,8,NaN,1.0,NaN
5288,Women's Clothing > Underwear & Sleepwear > Wom...,5288,0,0,NaN,NaN,NaN
5289,Women's Clothing > Underwear & Sleepwear > Wom...,5289,0,1,NaN,1.0,NaN



Class Overlap. In some cases, you may want to merge classes in the top rows (below)
-----------------------------------------------------------------------------------



,Class Name A,Class Name B,Class Index A,Class Index B,Num Overlapping Examples,Joint Probability
0,Women's Clothing > Tops > Tees,Men's Clothing > Tops & Tees > T-Shirts,729,3200,935,0.014606
1,Jewelry & Accessories > Rings,Jewelry & Accessories > Wedding & Engagement J...,2118,3121,403,0.006295
2,Women's Clothing > Activewear & Loungewear > H...,Men's Clothing > Hoodies & Sweatshirts,681,2122,391,0.006108
3,Apparel Accessories > Women's Accessories > Wo...,Security & Protection > Workplace Safety Suppl...,77,206,206,0.003218
4,Women's Clothing > Tops > Blouses & Shirts,Women's Clothing > Tops > Tees,726,729,183,0.002859
...,...,...,...,...,...,...
13994690,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Cookwar...",971,1418,0,0.000000
13994691,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Cookwar...",971,1419,0,0.000000
13994692,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Dessert...",971,1420,0,0.000000
13994693,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Dessert...",971,1421,0,0.000000



 * Overall, about 38% (24,510 of the 64,014) labels in your dataset have potential issues.
 ** The overall label health score for this dataset is: 0.62.

Generated with <3 from Cleanlab.



In [94]:
report_assume_appen2offshore = health_summary(labels=df.appen_category_path_labid, pred_probs=pred_probs_offshore, class_names=np.array(class_names), 
    asymmetric=False
)

---------------------------------------------------------------
|  Generating a Cleanlab Dataset Health Summary               |
|   for your dataset with 64,014 examples and 5,291 classes.  |
|  Note, Cleanlab is not a medical doctor... yet.             |
---------------------------------------------------------------



/opt/conda/envs/py38/lib/python3.8/site-packages/cleanlab/dataset.py:90: RuntimeWarning: invalid value encountered in divide
  given_conditional_noise = given_label_noise / joint.sum(axis=1)  # p(y!=k, s=k) / p(s=k)
/opt/conda/envs/py38/lib/python3.8/site-packages/cleanlab/dataset.py:91: RuntimeWarning: invalid value encountered in divide
  true_conditional_noise = true_label_noise / joint.sum(axis=0)  # p(s!=k, y=k) / p(y=k)


Overall Class Quality and Noise across your dataset (below)
------------------------------------------------------------ 



,Class Name,Class Index,Label Issues,Inverse Label Issues,Label Noise,Inverse Label Noise,Label Quality Score
0,Consumer Electronics > Sports & Action Video C...,2540,4,0,1.0,NaN,0.0
1,Beauty & Health > Skin Care > Hands & Nails > ...,3813,1,0,1.0,NaN,0.0
2,Home & Garden > Home Decor > Christian Decor >...,1127,1,0,1.0,NaN,0.0
3,Home Improvement > Bathroom Fixtures > Bathroo...,1758,1,2,1.0,1.0,0.0
4,Home Improvement > Bathroom Fixtures > Bathroo...,1757,1,0,1.0,NaN,0.0
...,...,...,...,...,...,...,...
5286,Home Improvement > Home Appliances > Major App...,5278,0,0,NaN,NaN,NaN
5287,Home Improvement > Home Appliances > Major App...,5279,0,0,NaN,NaN,NaN
5288,Home Improvement > Home Appliances > Major App...,5280,0,0,NaN,NaN,NaN
5289,Men's Clothing > Weddings & Events > Wedding P...,5284,0,0,NaN,NaN,NaN



Class Overlap. In some cases, you may want to merge classes in the top rows (below)
-----------------------------------------------------------------------------------



,Class Name A,Class Name B,Class Index A,Class Index B,Num Overlapping Examples,Joint Probability
0,Women's Clothing > Tops > Tees,Men's Clothing > Tops & Tees > T-Shirts,729,3200,935,0.014606
1,Jewelry & Accessories > Rings,Jewelry & Accessories > Wedding & Engagement J...,2118,3121,403,0.006295
2,Women's Clothing > Activewear & Loungewear > H...,Men's Clothing > Hoodies & Sweatshirts,681,2122,391,0.006108
3,Apparel Accessories > Women's Accessories > Wo...,Security & Protection > Workplace Safety Suppl...,77,206,205,0.003202
4,Women's Clothing > Tops > Blouses & Shirts,Women's Clothing > Tops > Tees,726,729,183,0.002859
...,...,...,...,...,...,...
13994690,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Cookwar...",971,1419,0,0.000000
13994691,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Dessert...",971,1420,0,0.000000
13994692,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Dessert...",971,1421,0,0.000000
13994693,Automobiles & Motorcycles > Auto Replacement P...,"Home & Garden > Kitchen,Dining & Bar > Dessert...",971,1422,0,0.000000



 * Overall, about 38% (24,544 of the 64,014) labels in your dataset have potential issues.
 ** The overall label health score for this dataset is: 0.62.

Generated with <3 from Cleanlab.



In [100]:
report_assume_appen2offshore['overlapping_classes'].head(10).tail(10).to_dict('records')

[{'Class Name A': "Women's Clothing > Tops > Tees",
  'Class Name B': "Men's Clothing > Tops & Tees > T-Shirts",
  'Class Index A': 729,
  'Class Index B': 3200,
  'Num Overlapping Examples': 935,
  'Joint Probability': 0.01460617989814728},
 {'Class Name A': 'Jewelry & Accessories > Rings',
  'Class Name B': 'Jewelry & Accessories > Wedding & Engagement Jewelry > Engagement Rings',
  'Class Index A': 2118,
  'Class Index B': 3121,
  'Num Overlapping Examples': 403,
  'Joint Probability': 0.006295497859843159},
 {'Class Name A': "Women's Clothing > Activewear & Loungewear > Hoodies & Sweatshirts",
  'Class Name B': "Men's Clothing > Hoodies & Sweatshirts",
  'Class Index A': 681,
  'Class Index B': 2122,
  'Num Overlapping Examples': 391,
  'Joint Probability': 0.006108038866497954},
 {'Class Name A': "Apparel Accessories > Women's Accessories > Women's Masks",
  'Class Name B': 'Security & Protection > Workplace Safety Supplies > Masks',
  'Class Index A': 77,
  'Class Index B': 206,
